<a href="https://colab.research.google.com/github/seungyeon38/Webtoon_recommendation_system/blob/master/TF-IDF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import string
string.punctuation
from math import log
import nltk
nltk.download('stopwords')
import re
from tqdm import tqdm


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [2]:
comic = pd.read_csv('./sample_data/Webtoon Dataset.csv')

In [23]:
#defining the function to remove punctuation
# 글자가 puctuation에 해당하면 " "를, 해당하지 않으면 글자 그대로
# print(string.punctuation)
def remove_punctuation(text):
    punctuationfree_list = []

    for word in text:
      if word in string.punctuation:
        punctuationfree_list.append(" ")
      else :
        punctuationfree_list.append(word)
    return "".join(punctuationfree_list)

# def remove_punctuation(text):
#     punctuationfree = "".join([i for i in text if i not in string.punctuation])
#     return punctuationfree

  # punctuationfree = "".join([i for i in text if i not in string.punctuation])


comic['Clean_Summary'] = comic['Summary'].apply(lambda x:remove_punctuation(x))

#setting lower case
comic['Lower_Summary'] = comic['Clean_Summary'].apply(lambda x: x.lower())

!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~


In [14]:
# #defining function for tokenization
# 토큰화
def tokenization(text):
    # print(text)
    tokens = re.split('\W+', text)
    # print(tokens)
    return tokens

#defining function for tokenization
# 영어만 단어단위로 토큰화
# p = re.compile('[a-z]+\d')

# def tokenization(text):
#     print(text)
#     result = p.findall(text)
#     print(result)
#     return result

#applying function to the column

comic['Tokenied_Summary'] = comic['Lower_Summary'].apply(lambda x: tokenization(x))


she s young  single and about to achieve her dream of creating incredible videogames  but then life throws her a one two punch  a popular streamer gives her first game a scathing review  even worse she finds out that same troublesome critic is now her new neighbor  a funny  sexy and all too real story about gaming  memes and social anxiety  come for the plot  stay for the doggo 
['she', 's', 'young', 'single', 'and', 'about', 'to', 'achieve', 'her', 'dream', 'of', 'creating', 'incredible', 'videogames', 'but', 'then', 'life', 'throws', 'her', 'a', 'one', 'two', 'punch', 'a', 'popular', 'streamer', 'gives', 'her', 'first', 'game', 'a', 'scathing', 'review', 'even', 'worse', 'she', 'finds', 'out', 'that', 'same', 'troublesome', 'critic', 'is', 'now', 'her', 'new', 'neighbor', 'a', 'funny', 'sexy', 'and', 'all', 'too', 'real', 'story', 'about', 'gaming', 'memes', 'and', 'social', 'anxiety', 'come', 'for', 'the', 'plot', 'stay', 'for', 'the', 'doggo', '']
after binge watching beauty videos

In [18]:
#stop words present in the library
stopwords = nltk.corpus.stopwords.words('english')
# print(stopwords[:20]) 

#defining the function to remove stopwords from tokenized text
# stopwords 제거 
def remove_stopwords(text):
    for word in text:
      output= [word for word in text if word not in stopwords]
    return output
    
#applying the function
comic['No_Stopwords_Summary'] = comic['Tokenied_Summary'].apply(lambda x:remove_stopwords(x))


['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his']


In [19]:
df1 = comic['No_Stopwords_Summary']
doc_words_list = df1.values.tolist()

docs = []

for doc_words in doc_words_list :
  docs.append(" ".join(doc_words))


# docs.shape
# type(docs) #pandas.core.frame.DataFrame
# df1.head()
# docs = [
#   '먹고 싶은 사과',
#   '먹고 싶은 바나나',
#   '길고 노란 바나나 바나나',
#   '저는 과일이 좋아요'
# ] 
# vocab = list(set(w for doc in docs for w in doc.split()))
vocab_list = list(set(w for doc in doc_words_list for w in doc))
vocab_list.sort()


In [20]:
docs_num = len(docs) # 총 문서의 수

def tf(t, d):
    return d.count(t)

def idf(t):
    df = 0
    for doc in docs:
        df += t in doc
    return log(docs_num/(df + 1))

def tfidf(t, d):
    return tf(t,d)* idf(t)

In [21]:
result = []
# total_result = [0 for i in range(len(vocab_list))] # 단어 개수 세려고 만든 리스트

for i in range(docs_num): # 각 문서에 대해서 아래 명령을 수행
    result.append([])
    doc = docs[i]
    for j in range(len(vocab_list)):
        vocab = vocab_list[j]        
        result[-1].append(tf(vocab, doc)) # 각 문서에서 각 단어의 수를 세어서 result list에 넣음 
        total_result[j] += tf(vocab, doc)

print(total_result)

# total_result_1_num = 0
# total_result_big_num = 0

# for i in total_result:
#   if i==1:
#     total_result_1_num += 1 
#   elif i>20:
#     total_result_big_num += 1
    
# print('total_result_1_num')
# print(total_result_1_num) # 2530 -> 2997

# print('total_result_big_num')
# print(total_result_big_num) # 481 -> 113

tf_ = pd.DataFrame(result, columns = vocab_list)
tf_
# tf_.shape #(569, 5696) -> (569, 5443)


[122322, 11, 47, 13, 9, 3, 1, 1, 3, 2, 1, 1, 1, 2, 1, 4, 1, 1, 7, 1, 1, 1, 1, 1, 42, 16, 1, 2, 1, 2, 1, 1, 3, 1, 4, 4, 1, 3, 3, 12, 3, 2, 1, 2, 1, 1, 1, 2, 2, 15, 2, 1, 5, 1, 5, 15, 2, 1, 18, 1, 1, 176, 5, 1, 1, 3, 10, 22, 78, 1, 1, 1, 2, 1, 3, 1, 2, 454, 7, 10, 3, 3, 1, 1, 3, 1, 1, 23, 10, 1, 1, 1, 1, 107, 6, 1, 1, 13, 105, 3, 14, 3, 1, 2, 6, 3, 2, 13, 10, 1, 1, 4, 1, 4, 1, 1, 1, 1, 2, 1, 1, 1, 3, 1, 1, 4, 1, 1, 7, 2, 1, 2, 1, 1, 30, 2, 14, 1, 1, 2, 1, 64, 6, 1, 2, 2, 5, 4, 1, 1, 1, 1, 1, 6, 1, 1, 1, 1, 2, 1, 106, 7, 1, 5, 23, 2, 26, 1, 5, 3, 251, 1, 2, 2, 4, 36, 2, 6, 2, 1, 1, 1, 2, 4, 100, 3, 10, 5, 4, 12, 1, 1, 1, 1, 4, 3, 101, 1, 4, 8, 24, 7, 4, 18, 5, 1, 1, 1, 4, 1, 1, 22, 2, 1, 1, 1, 2, 3, 1, 5, 1, 1, 1, 6, 1, 5, 2, 1, 1, 1, 3, 25, 2, 1, 1, 16, 1, 9, 6, 4, 1, 6, 1, 33, 8, 5, 1, 12, 7, 2, 1, 2, 3, 1, 1, 18, 5, 2, 6, 12, 1, 1, 32, 4, 6, 1, 1, 6, 18, 1, 3, 13, 8, 1, 11, 5, 85, 2, 1, 17, 7, 3, 5, 1, 2, 1, 1, 2, 4, 3, 1, 1, 1, 1, 1, 1, 20, 1, 1, 2, 1, 1, 18, 49, 3, 1, 1, 1, 5, 4, 1, 

,,000,1,10,100,1000,1001,10th,12,13,14,15,16th,17,1745,18,1842,18th,19,1935,1940s,1969,1989,19th,2,20,200,2020,2049,20s,20th,2111,21st,23,24,25,27,28,29th,3,...,yijae,yijun,yohan,yomanville,yong,yoo,york,young,younger,youngest,youth,youâ,yu,yuhyeon,yuli,yumi,yun,yunha,yunjo,yuri,yuseong,yuuki,yuzuki,zahn,zamsa,zealand,zecalion,zedekiah,zephyr,zero,zeronis,zodiac,zoe,zombie,zombies,zombified,zone,â,ã,ï
0,255,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,227,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,181,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,170,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,222,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
564,270,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
565,220,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
566,18,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
567,208,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [24]:
result = []
for j in range(len(vocab_list)):
    vocab = vocab_list[j]
    result.append(idf(vocab))

idf_ = pd.DataFrame(result, index = vocab_list, columns = ["IDF"])
idf_

,IDF
,-0.001756
000,3.945985
1,2.706294
10,3.778931
100,4.146656
...,...
zombified,5.650733
zone,5.650733
â,2.233007
ã,4.734443


In [25]:
result = []
for i in tqdm(range(docs_num)):
    result.append([])
    d = docs[i]
    for j in range(len(vocab_list)):
        vocab = vocab_list[j]
        result[-1].append(tfidf(vocab,d))

tfidf_ = pd.DataFrame(result, columns = vocab_list)
tfidf_

100%|██████████| 569/569 [08:36<00:00,  1.10it/s]


,,000,1,10,100,1000,1001,10th,12,13,14,15,16th,17,1745,18,1842,18th,19,1935,1940s,1969,1989,19th,2,20,200,2020,2049,20s,20th,2111,21st,23,24,25,27,28,29th,3,...,yijae,yijun,yohan,yomanville,yong,yoo,york,young,younger,youngest,youth,youâ,yu,yuhyeon,yuli,yumi,yun,yunha,yunjo,yuri,yuseong,yuuki,yuzuki,zahn,zamsa,zealand,zecalion,zedekiah,zephyr,zero,zeronis,zodiac,zoe,zombie,zombies,zombified,zone,â,ã,ï
0,-0.447761,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.139188,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,-0.398595,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,-0.317823,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.139188,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,-0.298508,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.778931,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,-0.389816,0.0,2.706294,3.778931,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
564,-0.474100,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
565,-0.386304,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
566,-0.031607,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
567,-0.365233,0.0,5.412589,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8.795941,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
print(result[500])
# print(len(result))

result_dictionary_list = []

# for i in range(len(result)):
#    result_dictionary = []
#    result_dictionary.append({k: v for v, k in enumerate(result[i])})
#    result_dictionary_list.append(result_dictionary)

# print(result_dictionary_list[0])

# print(dict(zip(result[0],range(len(result[0])))))


# for i in result[0]:
# print(sorted(result[0], reverse = True))


[1.3602738124179945, 0.0, 0.0, 5.6507332535663855, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 5.6507332535663855, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 5.6507332535663855, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,